# Импорты

In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Настройка отображения

In [30]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
plt.style.use('seaborn-v0_8-darkgrid')

# Загружаем данные

In [31]:
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')
sample_submission_df = pd.read_csv('data/sampleSubmission.csv')

# Базовая информация о датасетах

In [32]:
print("\n" + "=" * 80)
print("1. ОБЩАЯ ИНФОРМАЦИЯ О ДАТАСЕТАХ")
print("=" * 80)

print(f"\nОбучающая выборка (train.csv):")
print(f"Размер: {train_df.shape[0]} строк × {train_df.shape[1]} столбцов")
print(f"Период данных: {train_df['datetime'].min()} - {train_df['datetime'].max()}")

print(f"\nТестовая выборка (test.csv):")
print(f"Размер: {test_df.shape[0]} строк × {test_df.shape[1]} столбцов")
print(f"Период данных: {test_df['datetime'].min()} - {test_df['datetime'].max()}")

print(f"\nПример submission (sampleSubmission.csv):")
print(f"Размер: {sample_submission_df.shape[0]} строк × {sample_submission_df.shape[1]} столбцов")

# Детальная информация о столбцах
print("\n" + "=" * 80)
print("2. СТРУКТУРА ДАТАСЕТОВ")
print("=" * 80)

print("\nОбучающая выборка - первые 5 строк:")
print(train_df.head())

print("\n\nТестовая выборка - первые 5 строк:")
print(test_df.head())

print("\n\nПример submission - первые 5 строк:")
print(sample_submission_df.head())

# Информация о типах данных
print("\n" + "=" * 80)
print("3. ИНФОРМАЦИЯ О СТОЛБЦАХ И ТИПАХ ДАННЫХ")
print("=" * 80)

print("\nОбучающая выборка - информация о столбцах:")
print("-" * 50)
print(train_df.dtypes)

print("\n\nТестовая выборка - информация о столбцах:")
print("-" * 50)
print(test_df.dtypes)

# Статистика по числовым признакам (исключаем нечисловые)
print("\n" + "=" * 80)
print("4. СТАТИСТИЧЕСКОЕ ОПИСАНИЕ (train.csv)")
print("=" * 80)
# Выбираем только числовые столбцы для описания
numeric_cols = train_df.select_dtypes(include=[np.number]).columns
print(train_df[numeric_cols].describe().round(2))

# Проверка пропущенных значений
print("\n" + "=" * 80)
print("5. ПРОПУЩЕННЫЕ ЗНАЧЕНИЯ")
print("=" * 80)

print("\nОбучающая выборка:")
missing_train = train_df.isnull().sum()
print(missing_train[missing_train > 0] if missing_train.any() else "Пропущенных значений нет!")

print("\nТестовая выборка:")
missing_test = test_df.isnull().sum()
print(missing_test[missing_test > 0] if missing_test.any() else "Пропущенных значений нет!")

# Уникальные значения категориальных признаков
print("\n" + "=" * 80)
print("6. КАТЕГОРИАЛЬНЫЕ ПРИЗНАКИ")
print("=" * 80)

categorical_cols = ['season', 'holiday', 'workingday', 'weather']

for col in categorical_cols:
    print(f"\n{col}:")
    print(f"  Уникальные значения (train): {sorted(train_df[col].unique())}")
    print(f"  Уникальные значения (test):  {sorted(test_df[col].unique())}")
    print(f"  Частоты (train):")
    print(train_df[col].value_counts().sort_index())

# Анализ целевой переменной
print("\n" + "=" * 80)
print("7. АНАЛИЗ ЦЕЛЕВОЙ ПЕРЕМЕННОЙ (count)")
print("=" * 80)

print("\nСтатистика по количеству аренд:")
print(f"Минимум: {train_df['count'].min()}")
print(f"Максимум: {train_df['count'].max()}")
print(f"Среднее: {train_df['count'].mean():.2f}")
print(f"Медиана: {train_df['count'].median()}")
print(f"Стандартное отклонение: {train_df['count'].std():.2f}")

print("\nКорреляция с другими числовыми признаками:")
# Выбираем только числовые столбцы для корреляции
numeric_train_df = train_df.select_dtypes(include=[np.number])
corr_with_target = numeric_train_df.corr()['count'].sort_values(ascending=False)
print(corr_with_target)

# Анализ временных меток
print("\n" + "=" * 80)
print("8. ВРЕМЕННОЙ АНАЛИЗ")
print("=" * 80)

# Преобразуем даты
train_df['datetime_dt'] = pd.to_datetime(train_df['datetime'])
test_df['datetime_dt'] = pd.to_datetime(test_df['datetime'])

print(f"\nОбучающая выборка:")
print(f"Начало: {train_df['datetime_dt'].min()}")
print(f"Конец: {train_df['datetime_dt'].max()}")
print(f"Дней: {(train_df['datetime_dt'].max() - train_df['datetime_dt'].min()).days}")
print(f"Часов записей: {len(train_df)}")

print(f"\nТестовая выборка:")
print(f"Начало: {test_df['datetime_dt'].min()}")
print(f"Конец: {test_df['datetime_dt'].max()}")
print(f"Дней: {(test_df['datetime_dt'].max() - test_df['datetime_dt'].min()).days}")
print(f"Часов записей: {len(test_df)}")

# Создаем дополнительные признаки из даты
train_df['hour'] = train_df['datetime_dt'].dt.hour
train_df['dayofweek'] = train_df['datetime_dt'].dt.dayofweek
train_df['month'] = train_df['datetime_dt'].dt.month
train_df['year'] = train_df['datetime_dt'].dt.year

test_df['hour'] = test_df['datetime_dt'].dt.hour
test_df['dayofweek'] = test_df['datetime_dt'].dt.dayofweek
test_df['month'] = test_df['datetime_dt'].dt.month
test_df['year'] = test_df['datetime_dt'].dt.year

# Сводная информация для ElasticNet
print("\n" + "=" * 80)
print("9. СВОДКА ДЛЯ ElasticNet РЕГРЕССИИ")
print("=" * 80)

# Подсчет корреляции между temp и atemp
corr_temp_atemp = train_df['temp'].corr(train_df['atemp'])

# Выводим размерности для моделирования
print("\n" + "=" * 80)
print("РАЗМЕРНОСТИ ДЛЯ МОДЕЛИРОВАНИЯ")
print("=" * 80)

# Создаем X_train без нечисловых и целевых столбцов
exclude_cols = ['datetime', 'datetime_dt', 'casual', 'registered', 'count']
X_cols = [col for col in train_df.columns if col not in exclude_cols]

X_train = train_df[X_cols]
y_train = train_df['count']

print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")

print(f"\nПризнаки для моделирования:")
for i, col in enumerate(X_train.columns, 1):
    print(f"{i:2}. {col:15} | Тип: {X_train[col].dtype}")

print(f"\nПример X_train (первые 3 строки):")
print(X_train.head(3))

print(f"\nПример y_train (первые 5 значений):")
print(y_train.head())

# Показываем основные статистики корреляции
print("\n" + "=" * 80)
print("КЛЮЧЕВЫЕ КОРРЕЛЯЦИИ ДЛЯ МОДЕЛИРОВАНИЯ")
print("=" * 80)

key_correlations = {
    'temp - atemp': corr_temp_atemp,
    'temp - count': train_df['temp'].corr(train_df['count']),
    'atemp - count': train_df['atemp'].corr(train_df['count']),
    'humidity - count': train_df['humidity'].corr(train_df['count']),
    'hour - count': train_df['hour'].corr(train_df['count']),
}

for pair, corr_val in key_correlations.items():
    print(f"{pair:20} : {corr_val:.4f}")

print("\n" + "=" * 80)
print("ДАННЫЕ УСПЕШНО ЗАГРУЖЕНЫ И ПРОАНАЛИЗИРОВАНЫ!")
print("=" * 80)


1. ОБЩАЯ ИНФОРМАЦИЯ О ДАТАСЕТАХ

Обучающая выборка (train.csv):
Размер: 10886 строк × 12 столбцов
Период данных: 2011-01-01 00:00:00 - 2012-12-19 23:00:00

Тестовая выборка (test.csv):
Размер: 6493 строк × 9 столбцов
Период данных: 2011-01-20 00:00:00 - 2012-12-31 23:00:00

Пример submission (sampleSubmission.csv):
Размер: 6493 строк × 2 столбцов

2. СТРУКТУРА ДАТАСЕТОВ

Обучающая выборка - первые 5 строк:
              datetime  season  holiday  workingday  weather  temp   atemp  humidity  windspeed  casual  registered  count
0  2011-01-01 00:00:00       1        0           0        1  9.84  14.395        81        0.0       3          13     16
1  2011-01-01 01:00:00       1        0           0        1  9.02  13.635        80        0.0       8          32     40
2  2011-01-01 02:00:00       1        0           0        1  9.02  13.635        80        0.0       5          27     32
3  2011-01-01 03:00:00       1        0           0        1  9.84  14.395        75        0.0  